<div class="alert alert-block alert-success">
    <h1>DataLab HvA - serie Machine Learning Python - Tech report 4</h1>
</div>

<div class="alert alert-block alert-warning">
    <h2>Lineaire Regressie</h2>
</div>

### 0. Doel van deze les
Vanwege de serie lessen waarbinnen deze les valt is het hierin de insteek is om het gemaakte model te gaan vergelijken met andere andere Machine Learning modellen en daarbij een kwaliteitscriterium te hanteren waarmee *alle* modellen onderling vergelijkbaar zijn. Dit is de reden dat we werken met een test- en een trainset van onze data.
**Je leert** 
- hoe je in Python met behulp van het package `sklearn` een Lineair Regressiemodel kunt maken
- hoe je de geevalueerde modelcoëfficiënten van dit model kunt bekijken
- hoe je dit model kunt gebruiken om voorspellingen te genereren op nieuwe data

**Je leert niet:**
- algemene theoretisch wiskundige achtergronden over Lineaire Regressie, hiervoor verwijzen we naar een les statistiek in je eigen opleiding of één van de vele online tutorials hierover.
- bij meervoudige regressie: strategiën met betrekking tot het selecteren van de in het model te gebruiken variabelen (*feature selection*). 
- hoe je met het package `seaborn` eerste visuele verkenningen van de (variatie in de) data kan uitvoeren, zie hiervoor *Tech report 1: Introductie Machine Learning* uit deze serie.
- hoe je data importeert en inlaadt in de python sessie, zie hiervoor echter wel [deze instructie](https://www.youtube.com/watch?v=1oBVx7pyuXo&list=PLwkTCAI_gJjW10xaAJpdqeQx7EQgFolfG&index=10) 

**Machine Learning framework**
- Om een consistente leerlijn in de serie Machine Learning aan te houden kiezen we telkens voor een opzet waarin een inititeel model bepaald wordt o.b.v. een *deel* van de beschikbare data: de *train data*. Om het model te kunnen vergelijken met ándere modeltypen (waarbij andere algoritmen worden gebruikt) stellen we de voorspellende prestaties vast a.h.v. de nog niet gebruikte data: de *test data*. 

### 1. Laden van de benodigde packages
Packages die je zelf nog niet hebt geinstalleerd dien je vooraf in Python te installeren. 

In [ ]:
# Voor de dataset
import seaborn as sns

# Voor het bewerken en inspecteren van de data
import pandas as pd

# Voor het afsplitsen van een train- en test dataset
from sklearn.model_selection import train_test_split

# Voor het modelleren
from sklearn.linear_model import LinearRegression

### 2. Laden en prepareren van de data

We gebruiken als voorbeeld een dataset `'tips'`: "*One waiter recorded information about each tip he received over a period of a few months working in one restaurant. He collected several variables*". In totaal beschikken we over 244 waarnemingen. Lees eventueel meer over deze set [hier](https://rdrr.io/cran/reshape2/man/tips.html).   

#### Laden en weergave data

In [ ]:
# Laden van een standaard dataset uit package seaborn 
data = sns.load_dataset('tips')
print(data.head())
print(data.shape)
print(data.dtypes)

#### Dummy encoding categoriale variabelen
We zien dat we te maken hebben met een aantal categoriale variabelen, het regressiemodel kan hier niet mee overweg (zie ook de korte toelichting op het modeltype hieronder). We maken hiervoor *dummie variabelen* aan. Om ervoor te zorgen dat deze onderling niet volledig gecorreleerd zijn zal er één dummy minder worden aangemaakt dan er mogelijke waarden zijn (`drop_first=True`) :

In [ ]:
data = pd.get_dummies(data=data, drop_first=True)
print(data.head())
print(data.shape)
print(data.dtypes)

Voor de taak van aanmaken van Dummies of One-Hot-Encodig kan ook gebruik worden gemaakt van de scikit learn functies  `OneHotEncoder, LabelEncoder`. Echter ontstaan daarna objecte van het type `'numpy.ndarray'` (matrices) waarin bijvoorbeeld géén namen van de variabelen meer te vinden zijn. We geven hier de voorkeur aan de functie `get_dummies()` uit de `pandas` library waarin een panda's data frame wordt aangemaakt.

In het tech Report *TR04a - Lineaire Regressie-OneHotEncoder* is te zien hoe je met One-Hot encoding kan werken!

#### Split variabelen X en y
In dit voorbeeld willen we de waarde voor `tip` gaan voorspellen, zie ook *sectie 3: Voorbeschouwing* verderop.

In [ ]:
#independent variables / explanatory variables
X = data.drop('tip', axis=1)

#dependent variable / response / target variable.
y = data['tip']

#### Afsplitsen test data en train data
Voor meer details over deze stap, zie *Tech report 2: Datapreparatie* uit deze serie.

In [ ]:
# Split in training en test data 
# random_state kiezen zorgt voor reproduceerbaarheid van dit resultaat - gebruik dat ! Een andere waarde is uiteraard ook prima
# we kiezen voor een verhouden train - test van 70% - 30%. Andere keuzen kunnen ook!
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.30,random_state=999)
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

### 3. Voorbeschouwing
Zomaar modelleren doe je niet, eerst moeten we vaststellen wat we willen bereiken of welke vraag we zouden willen beantwoorden. Voor deze les zal dat worden:
- Taak: stel een lineair regressie model op om de hoogte van de fooi te voorspellen op basis van de overige variabelen 

Vanuit de taak zullen we dus stellen:
- De *onafhankelijke* variabelen ($x_i$) zijn de variabelen `total_bill`, `size`, `sex_Female`, `smoker_No`, `day_Fri`, `day_Sat`, `day_Sun`, `time_Dinner`, 
- De *afhankelijke* variabele ($y$) is de variabele `tip`

We zijn, met bovengenoemde definitie van $x_i$ en $y$ dus op zoek naar het model:
$$ y = \alpha + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_k x_k +\epsilon$$

Met in ons geval $k = 8$

Dat wil zeggen: we willen de *modelparameters* $\alpha$ en $\beta_i$ vaststellen. De term $\epsilon$ drukt de *fout* uit die elke voorspelling onvermijdelijk zal hebben. De algoritmen in `sklearn` die ons dit model gaan geven zorgen ervoor dat deze *fout* (eigelijk: het kwadraat ervan) voor de voorspellingen op de gebruikte traindata gemiddeld genomen zo klein mogelijk zal zijn. 

### 4. Model schatten
#### lees documentatie bij package
Voordat we een model gaan schatten is het van belang dat je de documentatie van het algoritme dat we uit het package gaan gebruiken lezen. In dit geval is dat dus voor `LinearRegression()`, die lees je bijvoorbeeld online [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Alternatief is om de `help()` functie te gebruiken, zie hieronder. 

In [ ]:
# We gaan de methode LinearRegression() uit het sklearn package gebruiken, inspecteer hiervoor de documentatie. Dat kan 
help(LinearRegression())

Zie bijvoorbeeld de parameter: `fit_intercept=True` (default waarde), als je die `=False` kiest zul je een $\alpha = 0$ afdwingen in het model. Ook lees je dat er een *method*  `fit()` voor deze *object class* gedefinieerd is om het model te *fitten*.

#### model fitten
We gebruiken dus de train data om een model te fitten. Dit is, zoals je ziet, een eenvoudig regeltje code. We geven het model een naam (`model_linreg`) zodat we het later weer opnieuw kunnen aanroepen / gebruiken.

In [ ]:
# Met model fitten default van package
model_linreg = LinearRegression().fit(train_X, train_y)

### 5. Model coëfficiënten en metrics van model zelf
We stellen vast welke modelcoëfficiënten $\alpha$ en $\beta_i$ het model voor ons heeft geschat. 

In [ ]:
# Model mét constante
# coefficiënt alpha
print("alpha =", model_linreg.intercept_)
# coefficient beta
print("[beta_1, beta_2, ..., beta_8] =", model_linreg.coef_)

### 6. Voorspellingen en perfomance op testset
Hieronder genereren we voorspellingen van y op de testset. Tevens produceren we een modelscore in termen van prestatiematen (performance measures, metrics) op de testset. Voor meer toelichting daarover verwijzen we naar *Tech report 3: performance metrics regressie en classificatie*

In [ ]:
predicted_y = model_linreg.predict(test_X)

In [ ]:
from sklearn import metrics
#MAE
print("DE MAE op de testset = ", metrics.mean_absolute_error(test_y, predicted_y))

#MSE
print("DE MSE op de testset = ", metrics.mean_squared_error(test_y, predicted_y))

#RMSE
print("DE RMSE op de testset = ", metrics.mean_squared_error(test_y, predicted_y, squared=False))

#MAPE
print("DE MAPE op de testset = ", metrics.mean_absolute_percentage_error(test_y, predicted_y))
